
# Degradazione Audio

Questo notebook scarica il dataset `fma_small` da [Free Music Archive](https://os.unil.cloud.switch.ch/fma/fma_small.zip), lo estrae, carica i file audio, applica rumore bianco per simulare una degradazione della qualità, e salva le versioni modificate in una directory separata.

## Passi:
1. Scaricare il dataset `fma_small`.
2. Estrarre i file audio.
3. Caricare i file audio.
4. Applicare rumore bianco.
5. Salvare le versioni modificate.



In [ ]:

import os
import urllib.request
import zipfile

# URL del dataset
url = "https://os.unil.cloud.switch.ch/fma/fma_small.zip"
dataset_path = "fma_small.zip"
extract_path = "fma_small"

# Scaricare il dataset
if not os.path.exists(dataset_path):
    print("Scaricamento del dataset...")
    urllib.request.urlretrieve(url, dataset_path)
    print("Dataset scaricato.")

# Estrarre il dataset
if not os.path.exists(extract_path):
    print("Estrazione del dataset...")
    with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Dataset estratto.")
else:
    print("Il dataset è già stato estratto.")


In [ ]:

import numpy as np
import librosa
import soundfile as sf

# Directory per salvare i file modificati
modified_path = "fma_small_modified"
os.makedirs(modified_path, exist_ok=True)

# Funzione per applicare rumore bianco
def add_white_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    augmented_audio = augmented_audio.astype(type(audio[0]))
    return augmented_audio

# Caricare i file audio, applicare rumore bianco e salvare le versioni modificate
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".mp3"):
            file_path = os.path.join(root, file)
            audio, sr = librosa.load(file_path, sr=None)
            modified_audio = add_white_noise(audio)
            modified_file_path = os.path.join(modified_path, file)
            sf.write(modified_file_path, modified_audio, sr)
            print(f"File modificato salvato: {modified_file_path}")
